# Software packages by researcher

This notebook uses the [DataCite GraphQL API](https://api.datacite.org/graphql) to fetch all software packages authored by a particular researcher, using his/her ORCID ID.

In [88]:
# Prepare the R graphql client.

library("httr")
library("ghql")
library("jsonlite")
library("IRdisplay")

cli <- GraphqlClient$new(
  url = "https://api.datacite.org/graphql"
)
qry <- Query$new()

In [89]:
# Generate the GraphQL query: find the researcher by ORCID ID, then fetch all software with DataCite DOIs linked to that ORCID account.

query <- '{
   researcher(id: "https://orcid.org/0000-0003-1444-9135") {
    name
    softwares {
      totalCount
      nodes {
        id
        creators {
          name
        }
        titles {
          title
        }
        publisher
        publicationYear
        relatedIdentifiers {
          relationType
          relatedIdentifier
        }
      }
    }
  }
}'

In [90]:
# Run the query and parse the JSON response

qry$query('getdata', query)
data <- fromJSON(cli$exec(qry$queries$getdata))

In [91]:
# Get the name of the researcher with ORCID ID https://orcid.org/0000-0003-1444-9135.

display_text(data$data$researcher$name)

Scott A Chamberlain


In [92]:
# Return the software packages as data frame, with the fields specified in the query

softwares <- data$data$researcher$softwares$nodes
softwares

id,creators,titles,publisher,publicationYear,relatedIdentifiers
<chr>,<list>,<list>,<chr>,<int>,<list>
https://doi.org/10.5281/zenodo.1183666,"Foster, Zachary , Chamberlain, Scott, Grunwald, Niklaus",Taxa V0.2.0,Zenodo,2017,"HasVersion , 10.5281/zenodo.1183667"
https://doi.org/10.5281/zenodo.1183667,"Foster, Zachary , Chamberlain, Scott, Grunwald, Niklaus",Taxa V0.2.0,Zenodo,2017,"IsVersionOf , 10.5281/zenodo.1183666"


In [94]:
# Generate a list of formatted citations in APA format for these software packages

ids <- softwares[,1]
ids <- substr(ids, 17, 100)
ids <- paste(ids, collapse = ',')
url <- paste('https://api.datacite.org/dois?style=apa&ids=', ids, sep = '')
response <- GET(url, accept("text/x-bibliography"))
display_markdown(content(response, as = 'text'))          

Foster, Z., Chamberlain, S., &amp; Grunwald, N. (2017, December 19). Taxa V0.2.0. Zenodo. https://doi.org/10.5281/ZENODO.1183666

Foster, Z., Chamberlain, S., &amp; Grunwald, N. (2017, December 19). Taxa V0.2.0. Zenodo. https://doi.org/10.5281/ZENODO.1183667